In [24]:
PATH_TO_DATA = 'c:/MaxTmp/Power2TheWiki/data/'

1) convert sql dump into csv file  
see https://stackoverflow.com/questions/27584405/how-to-import-a-mysqldump-into-pandas

In [17]:
from io import StringIO
import re, shutil
import os.path

reg1 = re.compile('`([\w_]+)`')
reg2 = re.compile(r'\((?P<val>\d+?,\'[a-z]*?\',\'.*?(?:(?:\\\\)|(?<!\\))\')\)')

FN_UK_LANGLINKS = 'ukwiki-20180620-langlinks.sql'
dump_filename = PATH_TO_DATA+FN_UK_LANGLINKS
target_table = "langlinks"

sio = StringIO()

read_mode = 0 # 0 - skip, 1 - header, 2 - data
with open(dump_filename, 'r', encoding='ISO-8859-1') as f:
    for line in f:
        line = line.strip()
        if line.lower().startswith('insert') and target_table in line:
            read_mode = 2
        if line.lower().startswith('create table') and target_table in line:
            read_mode = 1
            continue

        if read_mode==0:
            continue

        # Filling up the headers
        elif read_mode==1:
            if line.lower().startswith('primary'):
                # add more conditions here for different cases 
                #(e.g. when simply a key is defined, or no key is defined)
                read_mode=0
                sio.seek(sio.tell()-1) # delete last comma
                sio.write('\n')
                continue
            colheader = re.findall(reg1,line)
            for col in colheader:
                sio.write(col.strip())
                sio.write(',')

        # Filling up the data -same as @firelynx's code
        elif read_mode ==2:
            for match in reg2.finditer(line): 
                newline = match.group("val")
                #print(newline)
                newline = newline.strip(' ()')
                #print(newline)
                newline = newline.replace('`', '')
                #print(newline)
                sio.write(newline)
                sio.write("\n")
            #if line.endswith(';'):
                #print("END OF INSERT")
sio.seek(0)
with open (PATH_TO_DATA+'ukwiki-20180620-langlinks.csv', 'w', encoding='ISO-8859-1') as fd:
    shutil.copyfileobj(sio, fd,-1)

In [18]:
import pandas as pd

df_uk_langlinks = pd.read_csv(PATH_TO_DATA+'ukwiki-20180620-langlinks.csv', encoding='ISO-8859-1', quotechar="'", escapechar ="\\")
df_uk_langlinks

,ll_from,ll_lang,ll_title
0,25610,aa,Category:User de
1,25692,aa,Category:User de-1
2,159531,aa,Template:Babel
3,269084,aa,Template:Cc-by-sa-2.0
4,35964,aa,Template:Delete
5,1987209,aa,Template:FlowMention
6,244518,aa,Template:Style/IPA
7,244517,aa,Template:Style/yid
8,25620,aa,Template:User en-3
9,25723,aa,Template:User fr-1


In [20]:
df_uk_en_langlinks = df_uk_langlinks[df_uk_langlinks.ll_lang == 'en']
df_uk_en_langlinks.shape

(581098, 3)

In [23]:
import csv
df_uk_en_langlinks.to_csv(PATH_TO_DATA+'ukwiki-20180620-langlinks_en.csv', index = False, encoding='ISO-8859-1', quotechar="'", escapechar ="\\", quoting=csv.QUOTE_NONNUMERIC)